In [ ]:
from sklearn.datasets import load_breast_cancer
# Load the dataset
data = load_breast_cancer()
X = data.data  # features
y = data.target  # target labels (0 for malignant, 1 for benign)

# Optionally, display the description - uncomment the following line to do so
print(data.DESCR)

In [ ]:
import numpy as np

def sigmoid(x):
    z = np.clip( x, -500, 500 )           # protect against overflow
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def compute_cost_logistic(X, y, w, b, lambda_=0, safe=False):
    """
    Computes cost using logistic loss, non-matrix version

    Args:
      X (ndarray): Shape (m,n)  matrix of examples with n features
      y (ndarray): Shape (m,)   target values
      w (ndarray): Shape (n,)   parameters for prediction
      b (scalar):               parameter  for prediction
      lambda_ : (scalar, float) Controls amount of regularization, 0 = no regularization
      safe : (boolean)          True-selects under/overflow safe algorithm
    Returns:
      cost (scalar): cost
    """

    m,n = X.shape
    cost = 0.0
    for i in range(m):
        z_i    = np.dot(X[i],w) + b                                             #(n,)(n,) or (n,) ()
        if safe:  #avoids overflows
            cost += -(y[i] * z_i ) + log_1pexp(z_i)
        else:
            f_wb_i = sigmoid(z_i)                                                   #(n,)
            # Added small value to avoid log(0)
            cost += (-y[i] * np.log(f_wb_i + 1e-10) - (1 - y[i]) * np.log(1 - f_wb_i + 1e-10)) + (lambda_/(2*m))*np.dot(w,w)
    cost = cost/m

    reg_cost = 0
    if lambda_ != 0:
        for j in range(n):
            reg_cost += (w[j]**2)                                               # scalar
        reg_cost = (lambda_/(2*m))*reg_cost

    return cost + reg_cost

In [ ]:
import numpy as np

def calculate_gradient_for_loop(X, y, w, b, l):
    m, n = X.shape
    dw = np.zeros((n,))  # initialize the gradient vector
    db = 0.              # initialize the intercept gradient
    
    for i in range(m):
        z = np.dot(X[i], w) + b
        a = sigmoid(z)
        dz = a - y[i]
        for j in range(n):
            dw[j] += X[i][j] * dz + l/m * w[j]
        db += dz
    return dw / m, db / m


In [ ]:
import copy, math

def logistic_model(X, y, w_initial, b_initial, learning_rate=0.01, num_iterations=1000, l=0.0):
    J_history = []
    w = copy.deepcopy(w_initial)
    b = b_initial
    n = X.shape[1]
    
    for i in range(num_iterations):
        # Calculate the predicted values
        dw, db = calculate_gradient_for_loop(X, y, w, b, l)
        w = w - learning_rate * dw
        b = b - learning_rate * db
    
        for j in range(n):
            pass
        
            # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( compute_cost_logistic(X, y, w, b, l) )

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iterations / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}")    
    return w, b, J_history

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
w_tmp  = np.zeros_like(train_X[0])
b_tmp  = 0.
alpha = 0.1
iters = 1000
l = 0.7

w_out, b_out, history = logistic_model(train_X, train_y, w_tmp, b_tmp, alpha, iters, l) 

print(f"\nupdated parameters: w:{w_out}, b:{b_out}")

In [ ]:
from sklearn.metrics import accuracy_score

# Assume that `model` is your trained logistic regression model
# and `X_train` and `y_train` are your training data and labels

# Compute the model's predictions on the training data
# y_pred = model.predict(test_X)
z = np.dot(test_X, w_out) + b_out
predictions = sigmoid(z)
binary_predictions = (predictions > 0.5).astype(int)
# Compute the accuracy score
accuracy = accuracy_score(test_y, binary_predictions)

print(f"Accuracy: {accuracy}")

In [ ]:
def plot_data(X, y, ax, pos_label="y=1", neg_label="y=0", s=80, loc='best' ):
    """ plots logistic data with two axis """
    # Find Indices of Positive and Negative Examples
    pos = y == 1
    neg = y == 0
    pos = pos.reshape(-1,)  #work with 1D or 1D y vectors
    neg = neg.reshape(-1,)

    # Plot examples
    ax.scatter(X[pos, 0], X[pos, 1], marker='x', s=s, c = 'red', label=pos_label)
    ax.scatter(X[neg, 0], X[neg, 1], marker='o', s=s, label=neg_label, facecolors='none', edgecolors=dlblue, lw=3)
    ax.legend(loc=loc)

    ax.figure.canvas.toolbar_visible = False
    ax.figure.canvas.header_visible = False
    ax.figure.canvas.footer_visible = False

In [ ]:
import matplotlib.pyplot as plt
from plt_quad_logistic import plt_quad_logistic, plt_prob
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
print(w_out.shape)
w_out = w_out.reshape(-1, 1)
#print(w_out.shape)

w_pca = pca.fit_transform(w_out)
w_pca_flat = w_pca.flatten()

print(w_pca.shape)
fig,ax = plt.subplots(1,1,figsize=(5,4))
# plot the probability 

plt_prob(ax, w_pca, b_out)

# Plot the original data
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')   
ax.axis([0, 4, 0, 3.5])
plot_data(train_X,train_y,ax)

# Plot the decision boundary
x0 = -b_out/w_out[0]
x1 = -b_out/w_out[1]
ax.plot([0,x0],[x1,0], c=dlc["dlblue"], lw=1)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# Assume X is your input data and y are your labels
X = np.random.rand(100, 2)
y = np.random.randint(2, size=100)

# Fit a logistic regression model
model = LogisticRegression()
model.fit(X, y)

# Plot the data points
plt.scatter(X[:, 0], X[:, 1], c=y)

# Generate a grid over the input space to plot the color of the
#  classification at that grid point
xx, yy = np.mgrid[min(X[:,0])-1:max(X[:,0])+1:.01, min(X[:,1])-1:max(X[:,1])+1:.01]
grid = np.c_[xx.ravel(), yy.ravel()]
probs = model.predict_proba(grid)[:, 1].reshape(xx.shape)

# Plot the decision boundary
plt.contour(xx, yy, probs, levels=[.5], cmap="Greys", vmin=0, vmax=.6)

plt.show()